In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU, PReLU, ELU
from keras.layers import Dropout

import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv('../input/ccdata/churnArticle1.txt', sep=",")
df.head()
df.info()

In [ ]:
import numpy as np
df.describe(include=np.object)

In [ ]:
df.describe()

In [ ]:
#plotting the graph

import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(x = "Churn", data = df)
df.loc[:, 'Churn'].value_counts()

In [ ]:
# converting the non-numeric data into numeric data.
from sklearn.preprocessing import LabelEncoder
encoded = df.apply(lambda x: LabelEncoder().fit_transform(x) if x.dtype == 'object' else x)
encoded.head()
plt.figure(figsize =(20,20))
Corr=encoded[encoded.columns].corr()
sns.heatmap(Corr,annot=True)
X = encoded.iloc[:, 0:19]
y = encoded.Churn
X.head()
y.head()

In [ ]:
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

# fit an Extra Trees model to the data
model = ExtraTreesClassifier()
model.fit(X,y)
# display the relative importance of each attribute
print(model.feature_importances_)

# Feature Importance
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

# fit an Extra Trees model to the data
model = ExtraTreesClassifier()
model.fit(dataset.data, dataset.target)
# display the relative importance of each attribute
print(model.feature_importances_)

In [ ]:
from imblearn.over_sampling import SMOTE
k=1
seed=100
sm= SMOTE(sampling_strategy='auto', k_neighbors=k, random_state=seed)
X_res, y_res = sm.fit_resample(X, y)
X=X_res
y=y_res
#### [0-1] normalizasyon yöntemi
#from sklearn.preprocessing import Normalizer
#nm=Normalizer()
#X=nm.fit_transform(X)

## [-1,1] normalization
from sklearn.preprocessing import StandardScaler
SC = StandardScaler()
X = SC.fit_transform(X)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#print length of X_train, X_test, y_train, y_test
print ("X_train: ", len(X_train))
print("X_test: ", len(X_test))
print("y_train: ", len(y_train))
print("y_test: ", len(y_test))

In [ ]:
pip install keras_metrics

In [ ]:
import keras
import keras_metrics as km
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

def create_model(layers=(6, 3, 3),activation='relu', learn_rate=0.001,momentum=0.0, init='uniform'):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, kernel_initializer = init, activation=activation,input_dim = X_train.shape[1]))
            model.add(Dropout(0.1))
        else:
            model.add(Dense(nodes, kernel_initializer = init, activation=activation))
            model.add(Dropout(0.1))

    # Adding the output layer
    model.add(Dense(1, kernel_initializer = init, activation = 'sigmoid'))
    optimizer = SGD(lr=learn_rate, momentum=momentum)                    
    # Compiling the ANN
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[km.binary_f1_score(),
                                                                           km.binary_precision(),km.binary_recall(),"accuracy"])
    return model

In [ ]:
from time import time

In [ ]:
clf = KerasClassifier(build_fn=create_model, verbose=0)
layers = [(6, 3, 3), (10, 10), (45, 30, 15)]
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid']
batch_sizes=[10, 20, 40, 60, 80, 100]
epchos=[10, 50, 100]
learn_rate = [0.001, 0.1, 0.3, 0.6]
momentum = [0.0, 0.4, 0.8, 0.9]
init = ['uniform', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_random = dict(layers=layers, activation=activation,learn_rate=learn_rate, momentum=momentum, 
                    init=init, batch_size = batch_sizes, epochs=epchos)
model = RandomizedSearchCV(estimator = clf, param_distributions = param_random, 
                           n_iter = 5, cv = 10)

In [ ]:
# Utility function to report best scores
def report(results, n_top=10):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
start = time()
grid_result=model.fit(X_train, y_train)

print("RandomizedSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(model.cv_results_['params'])))

report(model.cv_results_)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

best_parameters = model.best_params_
best_accuracy = model.best_score_
print('Best Parameters after tuning: {}'.format(best_parameters))
print('Best Accuracy after tuning: {}'.format(best_accuracy))

In [ ]:
## RSM based approach - Experiments

In [ ]:
import keras
import keras_metrics as km
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(10, kernel_initializer = 'he_normal',activation='softsign',input_dim = 19))
#classifier.add(Dropout(0.1))

# Adding the second hidden layer
classifier.add(Dense(10, kernel_initializer = 'he_normal',activation='softsign'))
#classifier.add(Dropout(0.1))

# Adding the third hidden layer
#classifier.add(Dense(15, kernel_initializer = 'he_normal',activation='tanh'))
#classifier.add(Dropout(0.1))

# Adding the output layer
classifier.add(Dense(1, kernel_initializer = 'glorot_normal', activation = 'sigmoid'))

optimizer = SGD(lr=0.38, momentum=0.14) 
# Compiling the ANN
classifier.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=[km.binary_f1_score(),
                                                                           km.binary_precision(),km.binary_recall(),"accuracy"])

model_history = classifier.fit(X_train, y_train, validation_split=0.20, batch_size = 40, epochs = 50)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = classifier.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = classifier.predict_classes(X_test, verbose=0)

# predict probabilities for test set
yhat_probs = classifier.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = classifier.predict_classes(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]


# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %.4f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %.4f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %.4f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %.4f' % f1)
#roc-auc
roc_auc = roc_auc_score(y_test, yhat_classes)
print('ROC-AUC score: %.4f' % roc_auc)

In [ ]:
import xlwt 
  
workbook = xlwt.Workbook()  
  
sheet = workbook.add_sheet("Sheet3") 
  
# Specifying style 
style = xlwt.easyxf('font: bold 1') 
  
# Specifying column 
for i in range(20):
    # Initialising the ANN
    classifier = Sequential()

    # Adding the input layer and the first hidden layer
    classifier.add(Dense(10, kernel_initializer = 'he_normal',activation='softsign',input_dim = 19))
    #classifier.add(Dropout(0.1))

    # Adding the second hidden layer
    classifier.add(Dense(10, kernel_initializer = 'he_normal',activation='softsign'))
    #classifier.add(Dropout(0.1))

    # Adding the third hidden layer
    #classifier.add(Dense(15, kernel_initializer = 'he_normal',activation='tanh'))
    #classifier.add(Dropout(0.1))

    # Adding the output layer
    classifier.add(Dense(1, kernel_initializer = 'glorot_normal', activation = 'sigmoid'))

    optimizer = SGD(lr=0.40, momentum=0.14) 
    # Compiling the ANN
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=[km.binary_f1_score(),
                                                                           km.binary_precision(),km.binary_recall(),"accuracy"])

    model_history = classifier.fit(X_train, y_train, validation_split=0.20, batch_size = 40, epochs = 50)
    # predict probabilities for test set
    yhat_probs = classifier.predict(X_test, verbose=0)
    # predict crisp classes for test set
    yhat_classes = classifier.predict_classes(X_test, verbose=0)

    # predict probabilities for test set
    yhat_probs = classifier.predict(X_test, verbose=0)
    # predict crisp classes for test set
    yhat_classes = classifier.predict_classes(X_test, verbose=0)
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]


    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    precision = precision_score(y_test, yhat_classes)
    recall = recall_score(y_test, yhat_classes)
    f1 = f1_score(y_test, yhat_classes)
    roc_auc = roc_auc_score(y_test, yhat_classes)
    sheet.write(i, 0, accuracy, style)
    sheet.write(i, 1, precision, style)
    sheet.write(i, 2, recall, style)
    sheet.write(i, 3, f1, style)
    sheet.write(i, 4, roc_auc, style)
    workbook.save("set1_cc.xls") 